# Clustering  con K-means - Catado de cafe
Se tiene una base de datos con los resultados de diferentes catados de multiples muestras de cafe. 

Se desea realizar un agrupamiento de estas muestras segun sus metricas estadisticas por cada una de estas variables. Entre ellas se encuentra la calificacion promedio del catador certificado y niveles de sabor: Vainilla, floral, cereral, cocoa, alcohol, fermentado, tostado, oscuro, amargo, entre otros.

In [ ]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd

%matplotlib inline

## Importando Dataset y visualizando sus Caracteristicas

In [ ]:
cafes = pd.read_csv('../datasets/catacafe.csv',engine='python')
cafes.info()

## Mostrar las primeras filas para una previsualizacion del orden de los datos

In [ ]:
cafes.head()

## Se puede ver que la columna "Cafe" es el indice numerico de la muestra, pero que ya se ha enumerado con el metodo de lectura, por ello, se procede a eliminarla

In [ ]:
cafes = cafes.drop(['Cafe'],axis=1)
cafes.head()


## Se obtienen las variables estadisticas de los datos por columna

In [ ]:
cafes.describe()

## Se normalizan los datos en un rango adecuado y se vuelven a obtener sus metricas

In [ ]:
cafes_normalizado = (cafes - cafes.min())/(cafes.max()-cafes.min())
cafes_normalizado.describe()

Se puede observar que ahora los datos tienen valores entre cero y uno, max y min.
## Ahora con este preprocesamiento se tienen datos ordenados, numericos y normalizados, listos para un agrupamiento optimo.
El metodo de KMeans presenta gran efectividad y velocidad para datos no tan amplios, sin embargo, su principal debilidad es la seleccion de parametros de entrada, entiendase numero de clusters a realizar. Como este valor no se conoce, se debe usaralgun metodo para optimizar su implementacio, en este caso se utilizara el metodo del codo de Jambu para encontrar un numero de clusters optimo:

## Obtencion de grafica del codo de Jambu

Se desean que los clusteres sean lo mas separados entre si y que sus elementos sean lo mas cercanos entre si. para ello se Utiliza WCSS: la suma de los cuadrados de las distancias de cada punto de datos en todos los grupos a sus respectivos centroides, es decir, es una medida de similitud.
La idea es minimizar esta suma. Para ello se obtiene la inercia de cada clustering realizado con KMeans para un cierto numero de grupos, desde 1 hasta uno deseado, estos valores obtenidos en cada iteracion se almacena en WCSS, donde luego se imprimen en una grafica para su analisis.

In [ ]:
num_clusters = 10
wcss = []
for i in range(1,num_clusters+1):
    kmeans_model = KMeans(n_clusters=i,max_iter=300)
    kmeans_model.fit(cafes_normalizado)
    wcss.append(kmeans_model.inertia_)

## Ahora se grafican los resultados:

plt.plot(range(1,num_clusters+1),wcss)
plt.title("Codo de Jambu")
plt.xlabel("Numero de clusteres")
plt.ylabel("WCSS")
plt.show()

Se observa que le numero de clusteres optimo es 3, para este metodo y este dataset. Ahora se procede a utilizar el metodo Kmeans con este parametro. Igual que anteriormente, se crea el modelo de clustering y luego se aplica con .fit

In [ ]:
agrupamiento =  KMeans(n_clusters=3, max_iter=300) 
agrupamiento.fit(cafes_normalizado)

Este metodo crea un atributo label_ dentro del modelo clustering generado. Se agrega esta calificacion al archivo original del Dataset:

In [ ]:
cafes['KMeans_clusters'] = agrupamiento.labels_
cafes.head()

## Visualizacion de los clusters Generados
Los datos tienen multiples variables que los caracterizan, en este caso se desea visualizar un grafico lo mayor resumido posible, y se alcanza a visualizar hasta tres dimensiones.

Para efectos didacticos, se mostraran en dos dimensiones ¿Cuales? se seleccionan las variables que mejor caractericen a todos los datos, para ello se hace uso del Analaisis de Componentes Principales (PCA) para reducir el numero de variables a analizar, en este caso a visualizar.
Se hace uso del paquete descomposition de sklearn, se crea un dataframe a partir de estos componentes para graficarlo.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)                                   # Dos componentes principales
pca_cafes = pca.fit_transform(cafes_normalizado)
pca_cafes_df = pd.DataFrame(data= pca_cafes, columns=['Componente_1', 'Componente_2'])
pca_names_cafes = pd.concat([pca_cafes_df, cafes[['KMeans_clusters']]], axis=1)

pca_names_cafes   # veamos el resultado de los datos procesados:

## Graficar el dataframe procesado
Ahora se configura la figura plot a mostrar con estos datos obtenidos

In [ ]:
fig = plt.figure(figsize= (7,7))
grafico = fig.add_subplot(1,1,1)
grafico.set_xlabel('Componente 1',fontsize = 12 )
grafico.set_ylabel('Componente 2',fontsize = 12 )
grafico.set_title('Componentes principales - Clustering Kmeans',fontsize = 20 )
Colores = np.array(["blue", "orange", "green"])
grafico.scatter(x=pca_names_cafes.Componente_1, y=pca_names_cafes.Componente_2, c=Colores[pca_names_cafes.KMeans_clusters], s=40)
plt.show()

## Guardar los datos generados
Se procede a guardar el dataframe en formato csv:

In [ ]:
cafes.to_csv('../Results/cafe-kmeans.csv')   # Se crea un archivo csv en la carpeta Results

# Ejercicios/ Experimentos propuestos

## 1. Elija un valor aleatorio para el numero de clusters a implementar, suponiendo que no conoce el resultado del metodo codo de Jambu. ¿Como cambia el resultado? ¿Que se nota?
## 2. ¿Que sucede al aumentar o disminuir el numero de clusters a implementar en la llamada a KMeans? ¿Por que?
## 3. ¿Que sucede con los clusters al aumentar o disminuir significativamente el numero de iteraciones maximo (seteado en 300) al usar el metodo del Codo de Jambu y en la llamada a KMeans? ¿Por que?
## 4. Explique las ventajas y desventajas que tiene el algoritmo KMeans. Puede investigar diferentes fuentes.
## 5. Aumente el numero de PCA a 3 componentes y grafiquelo en 3 Dimensiones. ¿Que es lo que cambio y que se esta añadiendo?

## Experimentos a realizar
Aumentar las variables PCA a 3, configurando el grafico para mostrarlo.

In [ ]:
pca = PCA(n_components=3)
pca_cafes = pca.fit_transform(cafes_normalizado)
pca_cafes_df = pd.DataFrame(data= pca_cafes, columns=['Componente_1', 'Componente_2','Componente_3'])
pca_names_cafes = pd.concat([pca_cafes_df, cafes[['KMeans_clusters']]], axis=1)

pca_names_cafes   # veamos el resultado de los datos procesados:

In [ ]:
# Graficamos este nuevo dataframe en 3D
#from mpl_toolkits.mplot3d import Axes3D  #otra forma de plotearlo
#grafico = Axes3D(fig)

fig = plt.figure(figsize= (12,12))
grafico = fig.add_subplot(111, projection='3d')

x = pca_names_cafes.Componente_1
y = pca_names_cafes.Componente_2
z = pca_names_cafes.Componente_3

grafico.set_xlabel('Componente 1',fontsize = 12 )
grafico.set_ylabel('Componente 2',fontsize = 12 )
grafico.set_zlabel('Componente 3',fontsize = 12 )
grafico.set_title('3 Componentes principales - Clustering Kmeans',fontsize = 20 )
Colores = np.array(["blue", "orange", "green"])

# Agregamos los puntos en el plano 3D
grafico.scatter(x, y, z, c=Colores[pca_names_cafes.KMeans_clusters], marker='o')
plt.show()

# Como se puede observar, se esta agregando la tercer componente mas representativa del conjunto de datos, obtenida por medio de PCA, alcanzando la maxima visualizacion posible.
# Es posible mover el grafico 3D en algunos visualizadores y observar mejor la relacion entre los diferentes clusters.